In [ ]:
!pip install --upgrade langchain langchain-community
!pip install sentence-transformers torch
!pip install -qU langchain-text-splitters  
!pip install --upgrade --quiet qdrant-client 
!pip install pandas
!pip install matplotlib
!pip install tqdm

     |████████████████████████████████| 1.0 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 54.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 123.7 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 116.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 82.5 MB/s eta 0:00:01
     |████████████████████████████████| 746 kB 83.5 MB/s eta 0:00:01
     |████████████████████████████████| 397 kB 55.5 MB/s eta 0:00:01
     |████████████████████████████████| 431 kB 37.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 113.2 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 79.3 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.5 MB/s s eta 0:00:01
     |████████████████████████████████| 319 kB 101.3 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 105.2 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 108.8 MB/s eta 0:00:01
     |██████████████████████

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from typing import List, Dict
from tqdm import tqdm
import pyarrow.parquet as pq
import pyarrow as pa
import os
import gc
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant 
from langchain.schema import Document
import warnings
warnings.filterwarnings("ignore")

In [21]:
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("GPU is NOT available")

GPU is available
GPU name: NVIDIA GeForce RTX 4090


# 1. Chunk

In [ ]:
# Tạo text splitter với cấu hình có thể điều chỉnh
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=150,
    length_function=len,
    separators=[
        "\n\n",
        "\n1. ", "\n2. ", "\n3. ", "\n4. ", "\n5. ",
        "\na) ", "\nb) ", "\nc) ", "\nd) ", "\ne) ",
        "\n- ", "\n• ", "\n* ",
        "\n",
        ". ",
        " ",
        ""
    ]
)

In [23]:
def chunk_json_content(data: List[Dict], content_field: str = "content") -> List[Dict]:
    chunked_data = []

    for item in tqdm(data, desc="🔪 Chunking data"):
        if content_field not in item:
            chunked_data.append(item)
            continue

        content = item[content_field]

        if not content or content.strip() == "":
            chunked_data.append(item)
            continue

        chunks = text_splitter.split_text(content)

        for chunk in chunks:
            new_item = item.copy()
            new_item[content_field] = chunk
            chunked_data.append(new_item)

    return chunked_data

In [24]:
with open(r"/root/df.json", "r") as f:
    try:
        data = json.load(f)
    except json.JSONDecodeError as e:
        print("Lỗi JSON:", e)

In [25]:
# Tạo DataFrame từ dữ liệu JSON
df = pd.DataFrame(data)
# Tách riêng theo loại dữ liệu
df_article = df[df['type'] == 'article'].copy()
print(f"Số lượng bài viết (article): {len(df_article)}")
df_qa = df[df['type'] == 'qa'].copy()
print(f"Số lượng hỏi đáp (qa): {len(df_qa)}")

Số lượng bài viết (article): 21359
Số lượng hỏi đáp (qa): 14181


In [26]:
df_article = df_article.to_dict('records')
df_qa = df_qa.to_dict('records')

In [27]:
# Thực hiện chunking cho articles
print("\nBắt đầu chunking articles...")
chunked_data = chunk_json_content(df_article, content_field="content")
print(f"\nKết quả chunking:")
print(f"   - Trước chunking: {len(df_article)} articles")
print(f"   - Sau chunking: {len(chunked_data)} chunks")


Bắt đầu chunking articles...


🔪 Chunking data: 100%|██████████| 21359/21359 [00:02<00:00, 8518.53it/s]


Kết quả chunking:
   - Trước chunking: 21359 articles
   - Sau chunking: 149674 chunks


In [29]:
print(f"   - Tăng thêm: {len(chunked_data) - len(df_article)} chunks")

# Tính toán thống kê độ dài
original_lengths = [len(item.get('content', '')) for item in df_article]
chunk_lengths = [len(item.get('content', '')) for item in chunked_data]

print(f"\n Thống kê độ dài:")
print(f"   - Độ dài trung bình gốc: {sum(original_lengths)/len(original_lengths):.0f} ký tự")
print(f"   - Độ dài trung bình chunk: {sum(chunk_lengths)/len(chunk_lengths):.0f} ký tự")
print(f"   - Độ dài chunk max: {max(chunk_lengths)} ký tự")
print(f"   - Độ dài chunk min: {min(chunk_lengths)} ký tự")

   - Tăng thêm: 128315 chunks

 Thống kê độ dài:
   - Độ dài trung bình gốc: 5466 ký tự
   - Độ dài trung bình chunk: 795 ký tự
   - Độ dài chunk max: 1200 ký tự
   - Độ dài chunk min: 8 ký tự


In [30]:
filtered_data = [item for item in chunked_data if len(item["content"].strip()) >= 20]
print(f"Số lượng item sau khi lọc: {len(filtered_data)}")

Số lượng item sau khi lọc: 149650


In [31]:
# Lưu vào file JSON
with open("filtered_data.json", "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

print("Đã lưu filtered_data vào file 'filtered_data.json'")

Đã lưu filtered_data vào file 'filtered_data.json'


In [32]:
for i, item in enumerate(filtered_data[:10]):
    print(i)
    print(item['content'])

0
Căng cơ đùi có thể coi là một trong những chấn thương rất hay gặp, đặc biệt là ở những người bệnh thường xuyên chơi thể thao. Khi căng cơ ở đùi, bệnh nhân có thể gặp cảm giác đau đớn, khó chịu và khả năng vận động của bệnh nhân cũng bị ảnh hưởng, từ đó trực tiếp làm giảm chất lượng cuộc sống hàng ngày.
Bài viết này được viết dưới sự hướng dẫn chuyên môn của các bác sĩ thuộc khoa Chấn thương chỉnh hình & Y học thể thao Bệnh viện Đa khoa Quốc tế Vinmec.
1
1. Căng cơ đùi là như thế nào?
Cơ đùigồm ba nhóm chính:
Cơ tứ đầu đùi (Quadriceps): Mặt trước đùi, giúp duỗi gối.
Cơ gân kheo (Hamstring): Mặt sau đùi, giúp gập gối.
Cơ khép đùi (Adductors): Mặt trong đùi, giúp khép chân.
Căng cơ đùi là tình trạng các sợi cơ bị kéo giãn quá mức, gây tổn thương từ nhẹ đến nặng, có thể dẫn đến rách cơ một phần hoặc toàn bộ.
Chấn thương này thường xảy ra khi người bệnh hoạt động quá mức hoặc sai tư thế, đặc biệt trong các môn thể thao như bóng đá, chạy bộ, bóng rổ. Các động tác co duỗi chân đột ngột hoặc

# 2. Embedding

In [37]:
combined_data = filtered_data + df_qa
print(f"Tổng số item sau khi nối: {len(combined_data)}")
combined_data = pd.DataFrame(combined_data)

Tổng số item sau khi nối: 163831


In [38]:
combined_data

,url,title,content,tag,type
0,https://www.vinmec.com/vie/bai-viet/cang-co-du...,Căng cơ đùi: Nguyên nhân và các phương pháp đi...,Căng cơ đùi có thể coi là một trong những chấn...,Chấn thương chỉnh hình - Y khoa,article
1,https://www.vinmec.com/vie/bai-viet/cang-co-du...,Căng cơ đùi: Nguyên nhân và các phương pháp đi...,1. Căng cơ đùi là như thế nào?\nCơ đùigồm ba n...,Chấn thương chỉnh hình - Y khoa,article
2,https://www.vinmec.com/vie/bai-viet/cang-co-du...,Căng cơ đùi: Nguyên nhân và các phương pháp đi...,2. Nguyên nhân nào gây ra căng cơ đùi?\nCăng c...,Chấn thương chỉnh hình - Y khoa,article
3,https://www.vinmec.com/vie/bai-viet/cang-co-du...,Căng cơ đùi: Nguyên nhân và các phương pháp đi...,Bệnh lý thần kinh ngoại biên gây rối loạn trươ...,Chấn thương chỉnh hình - Y khoa,article
4,https://www.vinmec.com/vie/bai-viet/cang-co-du...,Căng cơ đùi: Nguyên nhân và các phương pháp đi...,3. Triệu chứng có thể gặp\nMột số dấu hiệu và ...,Chấn thương chỉnh hình - Y khoa,article
...,...,...,...,...,...
163826,https://www.vinmec.com/vie/bai-viet/lieu-luong...,Liều lượng uống thuốc Đông Y thế nào là hiệu q...,tôi đang có ý định sử dụng thuốc Đông y để điề...,Y học cổ truyền,qa
163827,https://www.vinmec.com/vie/bai-viet/nen-chon-t...,Nên chọn thuốc tự sắc hay thuốc sắc sẵn?,tôi đang sử dụng thuốc Đông Y để điều trị mất ...,Y học cổ truyền,qa
163828,https://www.vinmec.com/vie/bai-viet/uong-thuoc...,Uống thuốc đông y bao lâu khỏi đau dạ dày mãn ...,"Tôi bị đau dạ dày mãn tính, hiện đã bắt đầu sử...",Y học cổ truyền,qa
163829,https://www.vinmec.com/vie/bai-viet/thoi-diem-...,Thời điểm uống thuốc Đông Y hiệu quả nhất là t...,Tôi đã sử dụng thuốc Đông y được một thời gian...,Y học cổ truyền,qa


In [35]:
embeddings = HuggingFaceEmbeddings(
    model_name='strongpear/M3-retriever-MEDICAL',
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'batch_size': 64}
)

You try to use a model that was created with version 3.3.1, however, your version is 3.2.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [39]:
docs = [
    Document(
        page_content=f"{row['title']}\n\n{row['content']}",
        metadata={
            'url': row['url'],
            'title': row['title'],
            'tag': row['tag'],
            'type': row['type']
        }
    )
    for _, row in combined_data.iterrows()
]

In [ ]:
qdrant_cloud_api_key='QDRANT_API_KEY'
qdrant_url='QDRANT_URL'
qdrant_cloud = Qdrant.from_documents(
    docs,
    embeddings,
    url=qdrant_url,
    prefer_grpc=True,
    api_key=qdrant_cloud_api_key,
    collection_name="medical_data"
)

Embedding batches:  30%|███       | 390/1280 [05:01<11:28,  1.29it/s]


Saving file 0 with 50048 records...
✅ Saved embedded_data/embeddings_part_0.parquet - Size: 324.5 MB


Embedding batches:  61%|██████    | 781/1280 [10:10<06:25,  1.30it/s]


Saving file 1 with 50048 records...
✅ Saved embedded_data/embeddings_part_1.parquet - Size: 324.3 MB


Embedding batches:  92%|█████████▏| 1172/1280 [15:27<02:31,  1.40s/it]


Saving file 2 with 50048 records...
✅ Saved embedded_data/embeddings_part_2.parquet - Size: 324.1 MB


Embedding batches: 100%|██████████| 1280/1280 [18:59<00:00,  1.12it/s]
